### Read file data

In [ ]:
import pandas as pd

queries_train = pd.read_csv('/home/luungoc/BTL-2023.1/Deep learning/collectionandqueries/collection.tsv', sep='\t')

In [ ]:
queries_train.iloc[5]

In [ ]:
from tqdm import tqdm

collention = []

for i in tqdm(range(len(queries_train))):
    collention.append({
        'PID': queries_train.iloc[i][0],
        'Description': queries_train.iloc[i][1]
    })

In [ ]:
queries_train = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/Data Json/collection.json', 'r'))

In [ ]:
queries_train[1]

In [ ]:
from tqdm import tqdm

new_data = []
for item in tqdm(qrel_train):
    qid, pid = item['QID'], item['PID']
    
    new_data.append({
        'QID': qid,
        'PID': pid,
        'query': queries_train[qid]['PID'],
        'passage': collention[int(pid)]['Description']
    })

In [ ]:
import csv
import json

# Specify the input TSV file and output JSON file
tsv_file = '/home/luungoc/BTL-2023.1/Deep learning/collectionandqueries/qrels.train.tsv'



data = []
with open(tsv_file, 'r', encoding='utf-8', newline='') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in reader:
        data.append(row)

new_data = []

for item in data:
    new_data.append({
        'QID': item['QID'],
        'PID': item[None][1]
    })

with open(tsv_file.replace('tsv', 'json'), 'w', encoding='utf-8') as jsonfile:
    json.dump(new_data, jsonfile, indent=2)

print(f"Conversion complete. JSON file saved as {tsv_file.replace('tsv', 'json')}")


### Test dataloader

In [ ]:
from src.dataloader import MyDataset
from torch.utils.data import DataLoader

data_loader = MyDataset(data_path='/home/luungoc/BTL-2023.1/Deep learning/Data Json/data_total.json',
                        tokenizer='bert-base-uncased',
                        max_length=256)


train_data = DataLoader(dataset=data_loader, batch_size=8, num_workers=0)
data_loader[1]

In [ ]:
import pickle
import json


query_train = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/src/Data Json/query_train.json', 'r')) 
collection = json.load(open('/home/luungoc/BTL-2023.1/Deep learning/src/Data Json/collection.json', 'r')) 

In [ ]:
import pickle

# Mở file Pickle để đọc
with open('/home/luungoc/BTL-2023.1/Deep learning/src/Data Json/train_qrels_first_100k_with_negatives.pkl', 'rb') as file:
    # Sử dụng pickle.load để đọc dữ liệu từ file
    pair_train = pickle.load(file)

In [ ]:


from tqdm import tqdm
new_data = []

for item in tqdm(pair_train):
    temp = {}
    temp['negative_text'] = []
    for itm in item['hard_negative_pids'][:5]:
        temp['negative_text'].append({
            'Pid': itm,
            'P_text': collection[itm]
        })
    temp['query_text'] = {
        'Qid': item['qid'],
        'Q_text': query_train[str(item['qid'])]['PID'],
    }
    temp['positive_text'] = {
        'Pid' : item['pid'],
        'P_text': collection[item['pid']]
    }
    new_data.append(temp)
#     print(item['query_text'])

In [1]:
import torch.optim as optim
import torch
import argparse
import torch.nn as nn
from torch.utils.data import DataLoader
from model.cross_encoder import DoubleBert
from train_v2 import MyTrainer
from config import Configs
from dataloader import MyDataset


class args:
    config = 'config.ini'
    
config = Configs

data = MyDataset(config=config)

train_data = DataLoader(dataset=data, batch_size=2, num_workers=2, shuffle=True)
model = DoubleBert(config=config)

2023-12-13 21:32:17.787691: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# model = DoubleBert(config=config)

# state_dict = torch.load('/home/luungoc/BTL-2023.1/Deep learning/src/None_epoch_1_step_901.pth', map_location=torch.device('cpu') )

query = 'My name is Ngoc'
pos = 'Sky blue'

# state_dict = {key.replace('module.', ''): value for key, value in state_dict.items()}



# model.load_state_dict(state_dict, strict=False)
# Trước khi load state_dict, bạn có thể thực hiện bước này


model(query, pos)

(tensor([[0.9842]], grad_fn=<AddmmBackward0>),
 tensor([[0.5298, 0.4702]], grad_fn=<SoftmaxBackward0>))

In [5]:
a

tensor([[0.9056]], grad_fn=<AddmmBackward0>)

In [ ]:
import torch
import torch.nn as nn

class CrossEntropyLoss(nn.Module):
    def __init__(self):
        super(CrossEntropyLoss, self).__init__()

    def forward(self, softmax_positive, label_positive, softmax_negative, label_negative):
        # Chuyển đổi thành tensor và chuyển đổi thành kiểu Long
        label_positive = torch.tensor(label_positive, dtype=torch.long)
        label_negative = torch.tensor(label_negative, dtype=torch.long)

        # Tính toán CrossEntropy loss cho positive
        positive_loss = nn.CrossEntropyLoss()(softmax_positive, label_positive)

        # Tính toán CrossEntropy loss cho negative
        negative_loss = nn.CrossEntropyLoss()(softmax_negative, label_negative)

        # Tính trung bình loss
        mean_loss = (positive_loss + negative_loss) / 2

        return mean_loss

# Ví dụ sử dụng CrossEntropyLoss
ce_loss_criterion = CrossEntropyLoss()

# Giả sử bạn có các giá trị như mô tả
label_positive = [1, 1, 1, 1]
label_negative = [0, 0, 0, 0]
softmax_positive = torch.tensor([[0.1, 0.9], [0.2, 0.8], [0.3, 0.7], [0.25, 0.75]])
softmax_negative = torch.tensor([[0.1, 0.9], [0.2, 0.8], [0.3, 0.7], [0.25, 0.75]])

# Tính loss
mean_ce_loss = ce_loss_criterion(softmax_positive, label_positive, softmax_negative, label_negative)
print("Mean CrossEntropy Loss:", mean_ce_loss.item())


In [1]:
import torch.optim as optim
import torch
import argparse
import torch.nn as nn
from torch.utils.data import DataLoader
from model.cross_encoder import DoubleBert
from train_v2 import MyTrainer
from config import Configs
from dataloader import MyDataset


class args:
    config = 'config.ini'
    
config = Configs

data = MyDataset(config=config)

train_data = DataLoader(dataset=data, batch_size=2, num_workers=2, shuffle=True)
model = DoubleBert(config=config)

trainable_params_count = 0

no_decay = ["adapter.proj_up.bias", "adapter.proj_down.bias", "LayerNorm.bias"]
classifier = ["linear_layer.weight", "linear_layer.bias", "normalize.weight", "normalize.bias", "last_output.weight", "last_output.bias"]
layers = ["adapter.proj_up.weight", "adapter.proj_down.weight", "LayerNorm.weight"]

layers.extend(classifier)
# no_decay.extend(cls_bias)   

optimizer = [
    
    {
        "params": [p for n, p in model.named_parameters() if any([nd in n for nd in layers])],
        "weight_decay": 0.1,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },

]


for group in optimizer:
    for param in group["params"]:
        trainable_params_count += param.numel()
print(f'Total Trainable params: {trainable_params_count}')

2023-12-13 21:37:04.921866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were 

Total Trainable params: 10683650


In [2]:
for n, p in model.named_parameters():
    print(n)

model.embeddings.word_embeddings.weight
model.embeddings.position_embeddings.weight
model.embeddings.token_type_embeddings.weight
model.embeddings.LayerNorm.weight
model.embeddings.LayerNorm.bias
model.encoder.layer.0.attention.self.query.weight
model.encoder.layer.0.attention.self.query.bias
model.encoder.layer.0.attention.self.key.weight
model.encoder.layer.0.attention.self.key.bias
model.encoder.layer.0.attention.self.value.weight
model.encoder.layer.0.attention.self.value.bias
model.encoder.layer.0.attention.output.dense.weight
model.encoder.layer.0.attention.output.dense.bias
model.encoder.layer.0.attention.output.LayerNorm.weight
model.encoder.layer.0.attention.output.LayerNorm.bias
model.encoder.layer.0.attention.output.adapter.proj_down.weight
model.encoder.layer.0.attention.output.adapter.proj_down.bias
model.encoder.layer.0.attention.output.adapter.proj_up.weight
model.encoder.layer.0.attention.output.adapter.proj_up.bias
model.encoder.layer.0.intermediate.dense.weight
model.